# Enterprise Demo with Ollama

This notebook demonstrates clgraph's full capabilities using the enterprise-demo SQL pipeline with Ollama for LLM-powered features:

1. Load enterprise-demo SQL files (4-layer pipeline: raw → staging → analytics → marts)
2. Generate column descriptions using Ollama
3. Use the LineageAgent to answer natural language questions
4. Demonstrate text-to-SQL capabilities

**Requirements:**
- Install Ollama: `brew install ollama` (or see https://ollama.ai)
- Pull model: `ollama pull llama3.2`
- Start Ollama: `ollama serve`

### Configuration

Set the Ollama model and demo options here:

In [1]:
# Configuration - modify these as needed
OLLAMA_MODEL = "gpt-oss:20b"  # Or try: llama3.2, qwen2.5-coder:7b
SKIP_DESCRIPTIONS = False  # Set True to skip LLM description generation
SKIP_AGENT = False  # Set True to skip LineageAgent demo
SKIP_TEXT_TO_SQL = False  # Set True to skip text-to-SQL demo

### Imports and Setup

In [2]:
from pathlib import Path

from clgraph import Pipeline
from clgraph.agent import LineageAgent
from clgraph.column import generate_description
from clgraph.export import JSONExporter
from clgraph.tools import (
    ListTablesTool,
    SearchColumnsTool,
    TraceBackwardTool,
    TraceForwardTool,
)

# Path to ClickHouse example SQL files
CLICKHOUSE_EXAMPLE_SQL_PATH = Path(".") / "clickhouse_example"

### Load Enterprise Pipeline

Load the ClickHouse example SQL pipeline from the `clickhouse_example/` directory.

In [3]:
def load_enterprise_pipeline() -> Pipeline:
    """Load the ClickHouse example SQL pipeline."""
    if not CLICKHOUSE_EXAMPLE_SQL_PATH.exists():
        raise FileNotFoundError(
            f"ClickHouse example SQL files not found at: {CLICKHOUSE_EXAMPLE_SQL_PATH}\n"
            "Make sure you're running from the examples/ directory."
        )

    sql_files = sorted(CLICKHOUSE_EXAMPLE_SQL_PATH.glob("*.sql"))
    queries = []

    print("📂 Loading SQL files:")
    for sql_file in sql_files:
        if sql_file.name.startswith("00_"):
            continue
        content = sql_file.read_text()
        query_id = sql_file.stem
        queries.append((query_id, content))
        print(f"   ✓ {sql_file.name}")

    print()
    print("🔧 Creating pipeline with template_context={'env': 'dev'}...")

    pipeline = Pipeline.from_tuples(
        queries,
        dialect="clickhouse",
        template_context={"env": "dev"},
    )

    print(
        f"✅ Pipeline created: {len(pipeline.table_graph.tables)} tables, {len(pipeline.columns)} columns"
    )
    return pipeline


# Load the pipeline
print("🚀 Enterprise Demo with Ollama")
print()
pipeline = load_enterprise_pipeline()

🚀 Enterprise Demo with Ollama

📂 Loading SQL files:
   ✓ 01_raw_orders.sql
   ✓ 02_raw_customers.sql
   ✓ 03_raw_products.sql
   ✓ 04_raw_order_items.sql
   ✓ 10_staging_orders.sql
   ✓ 11_staging_customers.sql
   ✓ 12_staging_products.sql
   ✓ 20_analytics_customer_metrics.sql
   ✓ 21_analytics_product_metrics.sql
   ✓ 22_analytics_daily_sales.sql
   ✓ 30_marts_customer_360.sql
   ✓ 31_marts_sales_dashboard.sql

🔧 Creating pipeline with template_context={'env': 'dev'}...
Info: Skipping lineage for 01_raw_orders (no SELECT statement)
Info: Skipping lineage for 02_raw_customers (no SELECT statement)
Info: Skipping lineage for 03_raw_products (no SELECT statement)
Info: Skipping lineage for 04_raw_order_items (no SELECT statement)
✅ Pipeline created: 12 tables, 132 columns


### Setup Ollama LLM

Connect to Ollama for LLM-powered features.

In [4]:
def setup_ollama_llm(model: str = "llama3.2"):
    """Setup Ollama LLM for description generation."""
    try:
        from langchain_ollama import ChatOllama

        llm = ChatOllama(model=model, temperature=0.3)

        print(f"🔌 Testing Ollama connection (model: {model})...")
        response = llm.invoke("Say 'OK' if you can hear me.")
        print(f"✅ Ollama connected: {response.content[:50]}...")
        return llm

    except ImportError:
        print("❌ langchain-ollama not installed. Install with: uv pip install langchain-ollama")
        return None
    except Exception as e:
        print(f"❌ Failed to connect to Ollama: {e}")
        print("   Make sure Ollama is running: ollama serve")
        print(f"   And the model is pulled: ollama pull {model}")
        return None


# Setup LLM if needed
llm = None
if not (SKIP_DESCRIPTIONS and SKIP_AGENT and SKIP_TEXT_TO_SQL):
    llm = setup_ollama_llm(OLLAMA_MODEL)

🔌 Testing Ollama connection (model: gpt-oss:20b)...


✅ Ollama connected: OK...


### Direct Tool Usage (No LLM Required)

Demonstrate using lineage tools directly without an LLM.

In [5]:
print("=" * 80)
print("🔧 DIRECT TOOL USAGE (No LLM Required)")
print("=" * 80)
print()

# List tables
print("📋 ListTablesTool:")
print("-" * 60)
tool = ListTablesTool(pipeline)
result = tool.run()
print(f"Found {len(result.data)} tables:")
for table in result.data[:5]:
    print(f"  • {table['name']} ({table['column_count']} columns)")
print()

# Trace backward
print("🔙 TraceBackwardTool:")
print("-" * 60)
tool = TraceBackwardTool(pipeline)
result = tool.run(table="staging_dev.orders", column="amount")
print("Sources for staging_dev.orders.amount:")
print(result.message)
print()

# Trace forward
print("🔜 TraceForwardTool:")
print("-" * 60)
tool = TraceForwardTool(pipeline)
result = tool.run(table="raw_dev.orders", column="customer_id")
print("Dependents of raw_dev.orders.customer_id:")
print(result.message)
print()

# Search columns
print("🔍 SearchColumnsTool:")
print("-" * 60)
tool = SearchColumnsTool(pipeline)
result = tool.run(pattern="revenue")
print("Columns matching 'revenue':")
print(result.message)

🔧 DIRECT TOOL USAGE (No LLM Required)

📋 ListTablesTool:
------------------------------------------------------------
Found 12 tables:
  • analytics_dev.customer_metrics (12 columns)
  • analytics_dev.daily_sales (15 columns)
  • analytics_dev.product_metrics (15 columns)
  • marts_dev.customer_360 (21 columns)
  • marts_dev.sales_dashboard (18 columns)

🔙 TraceBackwardTool:
------------------------------------------------------------
Sources for staging_dev.orders.amount:
Column staging_dev.orders.amount is derived from: raw_dev.orders.total_amount

🔜 TraceForwardTool:
------------------------------------------------------------
Dependents of raw_dev.orders.customer_id:
Column raw_dev.orders.customer_id impacts: analytics_dev.customer_metrics.customer_id, marts_dev.sales_dashboard.daily_customers, marts_dev.sales_dashboard.mtd_customers, marts_dev.sales_dashboard.completion_rate, marts_dev.customer_360.valid_orders

🔍 SearchColumnsTool:
------------------------------------------------

### LLM Description Generation

Generate descriptions for columns that don't have them using Ollama.

In [6]:
if llm and not SKIP_DESCRIPTIONS:
    print("=" * 80)
    print("🤖 DESCRIPTION GENERATION")
    print("=" * 80)
    print()

    pipeline.llm = llm
    all_tables = sorted(pipeline.table_graph.tables.keys())

    # Count columns
    columns_with_desc = sum(1 for col in pipeline.columns.values() if col.description)
    columns_without_desc = len(pipeline.columns) - columns_with_desc

    print("📊 Pipeline Statistics:")
    print(f"   • Tables: {len(all_tables)}")
    print(f"   • Total columns: {len(pipeline.columns)}")
    print(f"   • With descriptions: {columns_with_desc}")
    print(f"   • Need generation: {columns_without_desc}")
    print()

    if columns_without_desc > 0:
        print("📝 Generating descriptions for first 5 columns without descriptions...")
        generated = 0
        for col in list(pipeline.columns.values())[:20]:
            if col.description is None and generated < 5:
                print(f"   • {col.table_name}.{col.column_name}...", end=" ", flush=True)
                try:
                    generate_description(col, llm, pipeline)
                    print("✓" if col.description else "✗")
                    generated += 1
                except Exception as e:
                    print(f"✗ {str(e)[:30]}")
        print(f"\n✅ Generated {generated} descriptions")
else:
    print("⏭️ Skipping description generation (SKIP_DESCRIPTIONS=True or no LLM)")

🤖 DESCRIPTION GENERATION

📊 Pipeline Statistics:
   • Tables: 12
   • Total columns: 132
   • With descriptions: 91
   • Need generation: 41

📝 Generating descriptions for first 5 columns without descriptions...
   • raw_dev.orders.order_id... 

✓
   • raw_dev.orders.customer_id... 

✓
   • raw_dev.orders.order_date... 

✓
   • staging_dev.orders.order_month... 

✓
   • raw_dev.orders.total_amount... 

✓

✅ Generated 5 descriptions


### LineageAgent Demo

Use natural language to query the lineage graph.

In [7]:
if llm and not SKIP_AGENT:
    print("=" * 80)
    print("🤖 LINEAGE AGENT (Natural Language Interface)")
    print("=" * 80)
    print()

    agent = LineageAgent(pipeline, llm=llm, verbose=True)

    print("Available tools:")
    for i, tool in enumerate(agent.list_tools(), 1):
        print(f"   {i:2}. {tool}")
    print()

    questions = [
        ("📋 Schema", "What tables are available?"),
        ("🔙 Backward", "Where does analytics_dev.customer_metrics.lifetime_value come from?"),
        ("🔍 Search", "Find columns containing 'customer'"),
    ]

    for category, question in questions:
        print(f"{category} ❓ {question}")
        print("-" * 70)
        result = agent.query(question)
        print(f"   🔧 Tool: {result.tool_used}")
        answer = result.answer[:200] + "..." if len(result.answer) > 200 else result.answer
        print(f"   📝 Answer: {answer}")
        print()
else:
    print("⏭️ Skipping LineageAgent demo (SKIP_AGENT=True or no LLM)")

🤖 LINEAGE AGENT (Natural Language Interface)

Available tools:
    1. trace_backward
    2. trace_forward
    3. get_lineage_path
    4. get_table_lineage
    5. list_tables
    6. get_table_schema
    7. get_relationships
    8. search_columns
    9. get_execution_order
   10. find_pii_columns
   11. get_owners
   12. get_columns_by_tag
   13. list_tags
   14. check_data_quality
   15. generate_sql
   16. explain_query

📋 Schema ❓ What tables are available?
----------------------------------------------------------------------
Question type: QuestionType.SCHEMA_TABLES
   🔧 Tool: list_tables
   📝 Answer: Found 12 tables (0 source, 12 derived)

🔙 Backward ❓ Where does analytics_dev.customer_metrics.lifetime_value come from?
----------------------------------------------------------------------
Question type: QuestionType.LINEAGE_BACKWARD
   🔧 Tool: trace_backward
   📝 Answer: Column analytics_dev.customer_metrics.lifetime_value is derived from: raw_dev.orders.total_amount

🔍 Search ❓ Fi

### JSON Export

Export the pipeline to JSON for persistence.

In [8]:
print("=" * 80)
print("💾 JSON EXPORT")
print("=" * 80)
print()

data = JSONExporter.export(pipeline, include_metadata=True, include_queries=True)

print("Exported pipeline:")
print(f"  • Dialect: {data['dialect']}")
print(f"  • Queries: {len(data['queries'])}")
print(f"  • Tables: {len(data['tables'])}")
print(f"  • Columns: {len(data['columns'])}")
print(f"  • Edges: {len(data['edges'])}")
print()

output_path = Path(".") / "enterprise_pipeline.json"
JSONExporter.export_to_file(pipeline, str(output_path), include_queries=True)
print(f"💾 Saved to: {output_path}")
print()

print("🔄 Testing round-trip...")
restored = Pipeline.from_json_file(str(output_path))
print(f"✅ Restored: {len(restored.table_graph.tables)} tables, {len(restored.columns)} columns")

print()
print("✅ Demo Complete!")

💾 JSON EXPORT

Exported pipeline:
  • Dialect: clickhouse
  • Queries: 12
  • Tables: 12
  • Columns: 132
  • Edges: 147

💾 Saved to: enterprise_pipeline.json

🔄 Testing round-trip...
Info: Skipping lineage for 01_raw_orders (no SELECT statement)
Info: Skipping lineage for 02_raw_customers (no SELECT statement)
Info: Skipping lineage for 03_raw_products (no SELECT statement)
Info: Skipping lineage for 04_raw_order_items (no SELECT statement)
✅ Restored: 12 tables, 132 columns

✅ Demo Complete!
